In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statistics import mean

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength']
n_cols = predictors.shape[1] # number of predictors
n_cols 

8

In [4]:
#A
#4.Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
mse_list=[]

for i in range(50):
    #1. Randomly split the data into a training and test sets by holding 30% of the data for testing.
    X_train, X_test, y_train, y_test = train_test_split(predictors,target,test_size=0.3)

    #2. Train the model on the training data using 50 epochs.
    #One hidden layer of 10 nodes, and a ReLU activation function
    #Use the adam optimizer and the mean squared error as the loss function.
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train,  y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)

    #3.Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.
    scores = model.evaluate(X_test, y_test, verbose=0)
    mse_list.append(mean_squared_error(y_test,model.predict(X_test)))

#5.Report the mean and the standard deviation of the mean squared errors.
print("Mean of MSE: {} \n STD of MSE: {}".format(mean(mse_list), np.std((mse_list))))

2023-05-31 11:21:18.768039: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-05-31 11:21:18.788219: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394325000 Hz
2023-05-31 11:21:18.788976: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5616f9e05510 executing computations on platform Host. Devices:
2023-05-31 11:21:18.789024: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-05-31 11:21:18.911202: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

Mean of MSE: 311.4351665923578 
 STD of MSE: 319.619785383807


In [5]:
#B.Normalize the data,use a normalized version of the data

predictors_norm = (predictors - predictors.mean()) / predictors.std()
mse_list=[]
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm,target,test_size=0.3)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train,  y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    mse_list.append(mean_squared_error(y_test,model.predict(X_test)))

print("Mean of MSE: {} \n STD of MSE: {}".format(mean(mse_list), np.std((mse_list))))

#How does the mean of the mean squared errors compare to that from Step A?

Mean of MSE: 364.3177293388075 
 STD of MSE: 95.1027009864037


In [8]:
#How does the mean of the mean squared errors compare to that from Step A?
print('Mean of the MSE increases, compare from step A.')

Mean of the MSE increases, compare from step A.


In [6]:
#C.Increase the number of epochs,use 100 epochs

predictors_norm = (predictors - predictors.mean()) / predictors.std()
mse_list=[]
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm,target,test_size=0.3)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train,  y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    mse_list.append(mean_squared_error(y_test,model.predict(X_test)))

print("Mean of MSE: {} \n STD of MSE: {}".format(mean(mse_list), np.std((mse_list))))

#How does the mean of the mean squared errors compare to that from Step A?




Mean of MSE: 170.38335221119573 
 STD of MSE: 21.121538172833056


In [9]:
#How does the mean of the mean squared errors compare to that from Step B?
print('Mean of the MSE decreases, compare from step B.')

Mean of the MSE decreases, compare from step B.


In [7]:
#D.Increase the number of hidden layers,Three hidden layers, each of 10 nodes and ReLU activation function.

predictors_norm = (predictors - predictors.mean()) / predictors.std()
mse_list=[]
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm,target,test_size=0.3)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train,  y_train, validation_data=(X_test, y_test), epochs=50, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    mse_list.append(mean_squared_error(y_test,model.predict(X_test)))

print("Mean of MSE: {} \n STD of MSE: {}".format(mean(mse_list), np.std((mse_list))))

#How does the mean of the mean squared errors compare to that from Step A?



Mean of MSE: 129.38494878490977 
 STD of MSE: 13.276164378490016


In [10]:
#How does the mean of the mean squared errors compare to that from Step B?
print('Mean of the MSE decreases, compare from step B.')

Mean of the MSE decreases, compare from step B.
